In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [9]:
data_folder = os.path.join(os.curdir, 'AllData')

train_path = os.path.join(data_folder, 'train')
val_path = os.path.join(data_folder, 'val')
test_path = os.path.join(data_folder, 'test')

train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [10]:
model = models.resnet18(pretrained=True)

num_classes = 11
model.fc = nn.Linear(model.fc.in_features, num_classes)

optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

model.to(DEVICE)

num_epochs = 20
best_val_acc = 0.9088


In [11]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    
    # Calculate validation accuracy and adjust learning rate (optional)
    model.eval()
    val_loss = 0.0
    val_correct = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
    
    val_acc = val_correct.double() / len(val_dataset)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] '
          f'Train Loss: {train_loss / len(train_dataset):.4f} '
          f'Val Loss: {val_loss / len(val_dataset):.4f} '
          f'Val Acc: {val_acc:.4f}')
    
    # Adjust learning rate (optional)
    scheduler.step(val_loss)
    
    # Save the best model weights
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

print('Training complete. Best validation accuracy:', best_val_acc)

Epoch [1/20] Train Loss: 1.7313 Val Loss: 1.0345 Val Acc: 0.7535
Epoch [2/20] Train Loss: 0.8097 Val Loss: 0.6180 Val Acc: 0.8556
Epoch [3/20] Train Loss: 0.5251 Val Loss: 0.4617 Val Acc: 0.8833
Epoch [4/20] Train Loss: 0.3974 Val Loss: 0.3975 Val Acc: 0.8986
Epoch [5/20] Train Loss: 0.3234 Val Loss: 0.3501 Val Acc: 0.8993
Epoch [6/20] Train Loss: 0.2527 Val Loss: 0.3250 Val Acc: 0.9030
Epoch [7/20] Train Loss: 0.2157 Val Loss: 0.3029 Val Acc: 0.9110
Epoch [8/20] Train Loss: 0.1723 Val Loss: 0.2894 Val Acc: 0.9103
Epoch [9/20] Train Loss: 0.1410 Val Loss: 0.2850 Val Acc: 0.9139
Epoch [10/20] Train Loss: 0.1179 Val Loss: 0.2764 Val Acc: 0.9110
Epoch [11/20] Train Loss: 0.0961 Val Loss: 0.2835 Val Acc: 0.9132
Epoch [12/20] Train Loss: 0.0764 Val Loss: 0.2775 Val Acc: 0.9117
Epoch [13/20] Train Loss: 0.0715 Val Loss: 0.2745 Val Acc: 0.9154
Epoch [14/20] Train Loss: 0.0613 Val Loss: 0.2769 Val Acc: 0.9139
Epoch [15/20] Train Loss: 0.0449 Val Loss: 0.2678 Val Acc: 0.9132
Epoch [16/20] Train

In [12]:
# Evaluate on test data
model.load_state_dict(torch.load('best_model.pth'))  # Load the best model
model.eval()

test_correct = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels.data)

test_acc = test_correct.double() / len(test_dataset)
print('Test Accuracy:', test_acc)

Test Accuracy: tensor(0.9107, device='cuda:0', dtype=torch.float64)
